In [1]:
import rebound as rb
import numpy as np
from celmech.nbody_simulation_utilities import set_time_step

Set up a simulation of the outer solar system

In [2]:
sim = rb.Simulation()
sim.units = ("Msun","AU","yr")
bodies = ["Sun","Jupiter","Saturn","Uranus","Neptune"]
for body in bodies:
    sim.add(body)
sim.move_to_com()
sim.integrator = 'whfast'
sim.ri_whfast.safe_mode=0
set_time_step(sim,1/20)

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')


Run integration, saving snapshots to a simulation archive file.

In [3]:

Tfin = 1e7
Nout = 512
save_file = "outer_solar_system_10Myr.sa"
sim.automateSimulationArchive(save_file,interval=Tfin/Nout,deletefile=True)

sim.integrate(Tfin)

Read in integration results

In [4]:
from celmech.nbody_simulation_utilities import get_simarchive_integration_results

results = get_simarchive_integration_results(save_file)

/Users/shadden/anaconda/envs/celmech/lib/python3.9/site-packages/rebound/simulationarchive.py:148: RuntimeWarning: The simulation might not be synchronized. You can manually synchronize it by calling sim.integrator_synchronize().
  warnings.warn("The simulation might not be synchronized. You can manually synchronize it by calling sim.integrator_synchronize().", RuntimeWarning)


Apply the FMFT algorithm to the planets' complex eccentricities, defined as
$$
z_i = e_i\exp(i\varpi_i)
$$

Given an input time series of $z(t)$ values at equally-spaced time intervals, the FMFT algorithm decomposes the time series into $N_\mathrm{freq}$ Fourier harmonics by determining frequencies $\nu_{j}$ and complex amplitudes, $a_j$, such that
$$
z_i(t) \approx \sum_{j=1}^{N_\mathrm{freq}}a_j\exp[i\nu_j t]
$$

In [5]:
from celmech.miscellaneous import frequency_modified_fourier_transform

# Calculate complex eccentricities
results['z'] = results['e'] * np.exp(1j * results['pomega'])

# Number of Fourier harmonics to calculate for each planet
Nfreq=2

time = results['time']
arc_sec_per_yr = 60*60*180 / np.pi


title="Frequency [\"/yr] | Amplitude"
print(title)
print("="*len(title))
for i,planet in enumerate(bodies[1:]):
    print(planet)
    print("-"*len(planet))


    # Calculate results
    z = results['z'][i]
    freq_results = frequency_modified_fourier_transform(time,z,Nfreq)
    
    
    # results are stored as a dictionary with entries
    # in the form: {nu_j:a_j}
    
    for freq,amp in freq_results.items():
        freq_scaled = freq * arc_sec_per_yr
        abs_amp = np.abs(amp)
        print("{:+.2f}\t\t|\t{:.4f}".format(freq_scaled, abs_amp))
    print("\n")

Frequency ["/yr] | Amplitude
Jupiter
-------
+4.25		|	0.0441
+28.23		|	0.0157


Saturn
------
+28.23		|	0.0482
+4.25		|	0.0337


Uranus
------
+4.25		|	0.0380
+3.09		|	0.0291


Neptune
-------
+0.67		|	0.0091
+3.09		|	0.0037


